In [7]:
# Cell 1: Setup & auto-detect latest landing folder
from pathlib import Path
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
project_root = Path(os.getenv("PROJECT_ROOT"))
landing_dir  = project_root / "landing"
subdirs      = [d for d in landing_dir.iterdir() if d.is_dir()]
if not subdirs:
    raise FileNotFoundError(f"No landing sub-folders under {landing_dir}")
data_dir     = max(subdirs, key=lambda d: d.stat().st_mtime)

print("⤷ Loading data from:", data_dir)


⤷ Loading data from: C:\Users\Earth\BEDROT PRODUCTIONS\BEDROT DATA LAKE\data_lake\landing\meta_ads_dump_20250527_131455


In [2]:
# Cell 2: Load raw JSON dumps
ads       = pd.read_json(data_dir / "ads.json")
adsets    = pd.read_json(data_dir / "adsets.json")
campaigns = pd.read_json(data_dir / "campaigns.json")
insights  = pd.read_json(data_dir / "insights.json")

# Convert key metrics to numeric
for col in ("spend","impressions","clicks","reach","cpc","ctr","frequency"):
    if col in insights.columns:
        insights[col] = pd.to_numeric(insights[col], errors="coerce")

print("Loaded shapes → ads:", ads.shape,
      "adsets:", adsets.shape,
      "campaigns:", campaigns.shape,
      "insights:", insights.shape)


Loaded shapes → ads: (54, 10) adsets: (31, 13) campaigns: (13, 7) insights: (31, 13)


In [3]:
# Cell 3: Flatten ads.json (including campaign_id & adset_id)
import json

ads_flat = ads.copy()
ads_flat["creative_id"] = ads_flat["creative"].apply(lambda c: c.get("id") if isinstance(c, dict) else None)
ads_flat["tracking_specs"] = ads_flat["tracking_specs"].apply(json.dumps)

ads_flat = ads_flat[[
    "id","campaign_id","adset_id","name","status","effective_status",
    "created_time","updated_time","creative_id","tracking_specs"
]].rename(columns={
    "id":"ad_id",
    "name":"ad_name"
})

print("ads_flat cols:", ads_flat.columns.tolist())


ads_flat cols: ['ad_id', 'campaign_id', 'adset_id', 'ad_name', 'status', 'effective_status', 'created_time', 'updated_time', 'creative_id', 'tracking_specs']


In [4]:
# Cell 4: Prepare campaigns_ & adsets_ for merge (drop campaign_id from adsets_)
campaigns_ = campaigns.rename(columns={
    "id":           "campaign_id",
    "name":         "campaign_name",
    "status":       "campaign_status",
    "objective":    "campaign_objective"
})

adsets_ = adsets.rename(columns={
    "id":              "adset_id",
    "name":            "adset_name",
    "status":          "adset_status",
    "daily_budget":    "adset_daily_budget",
    "lifetime_budget": "adset_lifetime_budget"
}).drop(columns=["campaign_id"])

print("campaigns_ cols:", campaigns_.columns.tolist())
print("adsets_ cols:",    adsets_.columns.tolist())


campaigns_ cols: ['campaign_id', 'campaign_name', 'campaign_status', 'campaign_objective', 'start_time', 'created_time', 'updated_time']
adsets_ cols: ['adset_id', 'adset_name', 'adset_status', 'adset_daily_budget', 'adset_lifetime_budget', 'bid_strategy', 'targeting', 'optimization_goal', 'start_time', 'pacing_type', 'created_time', 'updated_time']


In [5]:
# Cell 5: Build the tidy DataFrame
tidy = (
    insights
      .merge(campaigns_, on="campaign_id", how="left")
      .merge(adsets_,    on="adset_id",    how="left")
      .merge(ads_flat,   on=["campaign_id","adset_id"], how="left")
)

# Drop any duplicated columns and inspect
tidy = tidy.loc[:, ~tidy.columns.duplicated()]
print("Tidy shape:", tidy.shape)
tidy.head(3)


Tidy shape: (54, 38)


,campaign_id,campaign_name_x,adset_id,adset_name_x,spend,impressions,clicks,cpc,ctr,reach,...,created_time_y,updated_time_y,ad_id,ad_name,status,effective_status,created_time,updated_time,creative_id,tracking_specs
0,120224907266750080,PIG1987 - THE STATE OF THE WORLD - BROAD,120224907266740080,THE STATE OF THE WORLD - AD 1,3.41,1321,8,0.426250,0.605602,1299,...,2025-05-23T06:58:12-0700,2025-05-23T21:58:37-0700,120224907266800080,AD1,ACTIVE,ADSET_PAUSED,2025-05-23T06:58:13-0700,2025-05-23T07:01:41-0700,24188788294059211,"[{""action.type"": [""offsite_conversion""], ""fb_p..."
1,120224907266750080,PIG1987 - THE STATE OF THE WORLD - BROAD,120224907266810080,THE STATE OF THE WORLD - AD 3,19.43,11562,146,0.133082,1.262757,10146,...,2025-05-23T06:58:01-0700,2025-05-23T06:58:01-0700,120224907266760080,AD3,ACTIVE,ACTIVE,2025-05-23T06:58:02-0700,2025-05-23T07:02:14-0700,566770032783189,"[{""action.type"": [""offsite_conversion""], ""fb_p..."
2,120224907266750080,PIG1987 - THE STATE OF THE WORLD - BROAD,120224907266820080,THE STATE OF THE WORLD - AD 4,8.33,4238,51,0.163333,1.203398,4082,...,2025-05-23T06:58:16-0700,2025-05-24T21:09:19-0700,120224907266770080,AD4,ACTIVE,ADSET_PAUSED,2025-05-23T06:58:17-0700,2025-05-23T07:05:35-0700,2178986529195170,"[{""action.type"": [""offsite_conversion""], ""fb_p..."


In [6]:
# Cell 6: Export tidy DataFrame to CSV (to staging folder)
staging_dir = project_root / "staging"
staging_dir.mkdir(parents=True, exist_ok=True)

out_path = staging_dir / "tidy_metaads.csv"
tidy.to_csv(out_path, index=False)

print("✔ Wrote tidy CSV to:", out_path)


✔ Wrote tidy CSV to: C:\Users\Earth\BEDROT PRODUCTIONS\BEDROT DATA LAKE\data_lake\staging\tidy_metaads.csv
